In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json

def get_lyric(song_id, lv=True, tv=True):
    # lv显示原文，tv显示译文
    lv_ = '&lv=1' if lv else ''
    tv_ = '&tv=1' if tv else ''
    
    # 请求歌词报文
    url = f'http://music.163.com/api/song/lyric?id={song_id}{lv_}{tv_}'
    res = requests.get(url).text
    
    # 清洗歌词
    obj = json.loads(res)
    lyric = obj['lrc']['lyric'] if lv else ''
    tlyric = obj['tlyric']['lyric'] if tv else ''
    
    return lyric, tlyric

def get_songs(id, by='artist'):
    # 请求 by = artist | playlist
    url = f'https://music.163.com/{by}?id={id}'
    res = requests.get(url).text

    # 解析遍历
    songs = {}
    soup = BeautifulSoup(res)
    for li in soup.find('div', id='song-list-pre-cache').ul:
        name = li.a.string
        song_id = re.search('\d+', li.a['href']).group()
        songs.update({name:song_id})

    return songs

In [6]:
# 20689 MY FIRST STORY
# 20878 ONE OK ROCK
songs = get_songs('20878')

# 便利每首歌
for name, id in songs.items():
    # 原文、译文用相同逻辑处理
    for i, each in enumerate(get_lyric(id)):
        # 去掉[]及内其内容
        text = re.sub('\[.*\]', '', each)
        # 保存
        with open(f'./{i}/{name.replace("/","_")}.txt', 'w', encoding='utf-8') as f:
            f.write(text)

In [55]:
from pathlib import Path

# ./1译文，./0原文
p = Path('./1')

# 将所有文件放入一个字符串
text = ''
for each in p.iterdir():
    text += each.read_text()

# 中文间空格替换为回车，
text = re.sub('[\u4e00-\u9fa5]( )[\u4e00-\u9fa5]', '\n', text)

# 去除空行
while '\n\n' in text:
    text = text.replace('\n\n', '\n')


# 保存
Path('data.txt').write_text(text)

48521

In [5]:
from pathlib import Path

# ./1译文，./0原文
p = Path('./1')

# 将歌词存为列表
data = []
for each in p.iterdir():
    text = each.read_text()

    # 中文间空格替换为回车，去除空行
    text = re.sub('[\u4e00-\u9fa5]( )[\u4e00-\u9fa5]', '\n', text)
    while '\n\n' in text:
        text = text.replace('\n\n', '\n')

    data.append(text)

# 保存列表到文件
json.dump(data, Path('train.json').open('w'), ensure_ascii=False)